In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('Cell_a.csv')

In [10]:
pd.DataFrame(data.groupby('Job_ID').count().Task_ID)

,Task_ID
Job_ID,
375000667413,1
375000669289,1
375000670586,1
375000670588,1
375000670590,1
...,...
400465207745,1
400465219425,2
400465256347,1
